# Script Prep.

In [2]:
try:
  from google.colab import drive
  drive.mount('/content/drive/')
  shared_drive_foldername = 'NUTRECON'
  root = '/content/drive/Shareddrives/{}/'.format(shared_drive_foldername)
  %pip install pandas==1.4.2
  print('Running Code in Colab')
except:
  root = 'D:/FV/Projects/NUTRECON/nutreconDrive/'
  print('Running Code locally')

import sys
sys.path.append(root + '1. Acquisition/notebooks')
# sys.path.append(root + '1. Acquisition/notebooks_psychophysics')

from nutrecon import *

import numpy as np
from glob import glob
from datetime import datetime
import json

from random import shuffle, sample
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import seaborn as sns

_v_ = Vars()


Running Code locally
Running Code locally
Running Code locally


# 1. Flavor tasting order 

In [3]:
subject_nr = 3    # Change subject number here

# ==============================================================================

subject_code = '{}{:03d}'.format(_v_.experiment_code, subject_nr)
presentation_order_d2_df, _ = loadResponses(_v_.sequences_dataPath, _v_.pres_order_d2_fileID, subject_code)
presentation_order_d2_df.T

,Flavor,image,flavor_id,image_id
0,ginseng,cross,i,Image VI
1,cashew,square,c,Image I


## 1.1 Check ratings

In [4]:
pleas_min = 0
pleas_max = 80
df_pleas = None
df_pleas_ = None

ratings_colNames = ['User','Trial',_v_.novelty_colName, _v_.intensity_colName, _v_.pleasanteness_colName]
df_, _ = loadResponses(_v_.responses_dataPath, _v_.flav_ratings_fileID, subject_code)
if not df_ is None:
  tmp_df = presentation_order_d2_df.T
  tmp_df['Trial'] = np.arange(1,len(tmp_df)+1)
  df = tmp_df.merge(df_[ratings_colNames],
                                  left_on = 'Trial', right_on = 'Trial').drop(columns='Trial')
  df_pleas = df[(df[_v_.pleasanteness_colName] >= pleas_min) & (df[_v_.pleasanteness_colName] < pleas_max)].sort_values(by=_v_.novelty_colName, ascending = False)
  if len(df_pleas) == 0 :
    df_pleas = df.sort_values(by=[_v_.pleasanteness_colName, _v_.novelty_colName], key=abs).set_index(np.arange(1,len(df)+1))
    df_pleas['criteria'] = 'not in complience'
  else:
    df_pleas['criteria'] = 'in complience'
    flavors_ = df_pleas[_v_.flavorName_colName].to_list()
    df_ = df[~df[_v_.flavorName_colName].isin(flavors_)].sort_values(by=[_v_.pleasanteness_colName, _v_.novelty_colName], key=abs)
    df_['criteria'] = 'not in complience'
    df_pleas = pd.concat([df_pleas, df_])
  df_pleas = df_pleas.set_index(np.arange(len(df_pleas)) + 1).rename_axis('').reset_index().set_index(['criteria', ''])
  df_pleas_ = df_pleas.drop(columns = ['User'] )

df_pleas_

,,Flavor,image,flavor_id,image_id,novelty,intensity,pleasantness
criteria,,,,,,,,
in complience,1,ginseng,cross,i,Image VI,29,9,3
not in complience,2,cashew,square,c,Image I,26,24,-4


# 3. Reward realization (day 2)

In [6]:
df, responses_path = loadResponses(_v_.responses_dataPath, _v_.neuroEcon_responses_fileID, subject_code)

seed = int(responses_path.split('_')[-1].split('.')[0])
rng = np.random.default_rng(int(responses_path.split('_')[-1].split('.')[0]))

df_clean = df[df['choice'] != 0]

mix_yogurt_df = df_clean[df_clean['Trial Type'] == 'mixed_yogurt']
st_mt_df = df_clean[df_clean['Trial Type'] != 'mixed_yogurt']

realizationTrials_df  = pd.concat([mix_yogurt_df.sample(1), st_mt_df.groupby(['Trial Type', 'lottery type']).sample(1)]).reset_index(drop = True)

realizationResults_df = realizationTrials_df.apply(lambda row: realizeChoices(row,rng), axis = 1)
realizationResults_df = save_json(realizationResults_df, subject_code, _v_.neurEconRealization2_fileID , _v_.sequences_dataPath)
realizationResults_df[['Trial Type', 'reference type', 'lottery type', 'reference qt', 'lottery qt', 'reference p', 'lottery p', 'choice', 'reward Qt.',	'reward description']]

File already exists. Created on 2022-12-09 11:02:24


,Trial Type,reference type,lottery type,reference qt,lottery qt,reference p,lottery p,choice,reward Qt.,reward description
0,mixed_yogurt,C+,C-,40.0,120,0.75,0.13,1,40,mL of ginseng yogurt
1,mixed,money,C+,0.2,200,1.00,0.13,2,0,You got nothing
2,mixed,money,C-,0.2,80,1.00,0.22,2,0,You got nothing
3,same,C+,C+,20.0,20,1.00,0.22,1,20,mL of ginseng yogurt
4,same,C-,C-,20.0,40,1.00,0.75,1,20,mL of cashew yogurt
5,same,money,money,1.0,2,1.00,0.13,1,1,€
